In [ ]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip install xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 76.9 MB/s eta 0:00:00


In [ ]:
# %%capture
# !pip install unsloth
# # Also get the latest nightly Unsloth!
# !pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
# !export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [ ]:
# import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
# !pip install -Uq flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 73.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Operation cancelled by user

In [ ]:
# !export TORCH_CUDA_ARCH_LIST="7.0;8.0"  # Replace with your architecture(s)
# !pip install flash-attn --no-build-isolation

  Using cached flash_attn-2.6.3.tar.gz (2.6 MB)
  Preparing metadata (setup.py) ... done
ERROR: Operation cancelled by user


In [ ]:
# !pip install -q transformers==4.44.2
# !pip install -Uq -q accelerate peft bitsandbytes trl dataset bitsandbytes
# !pip install -q -U accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver do

In [ ]:
# !wget -c -q "https://huggingface.co/datasets/liuhaotian/LLaVA-Instruct-150K/resolve/main/llava_instruct_150k.json"

In [ ]:
# -*- coding: utf-8 -*-
"""prepare_dataset_tokenise.py - Optimized for Multimodal Fine-tuning"""

import os
import numpy as np
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, WhisperProcessor, WhisperForConditionalGeneration, PreTrainedModel,BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, TaskType
from datasets import Dataset, DatasetDict
from tqdm import tqdm
import json
import librosa
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import gc

# Initialize Whisper components for audio transcription
whisper_model_name = "openai/whisper-small"
whisper_processor = WhisperProcessor.from_pretrained(whisper_model_name)
whisper_model = WhisperForConditionalGeneration.from_pretrained(whisper_model_name)

# Load embeddings with error handling
def load_embeddings(file_path):
    try:
        data = np.load(file_path)
        if 'image_ids' in data and 'embeddings' in data:
            return {'ids': data['image_ids'], 'embeddings': data['embeddings']}
        else:
            raise ValueError(f"Unexpected structure in {file_path}.")
    except Exception as e:
        print(f"Error loading embeddings: {e}")
        return None

# Process audio files
def transcribe_speech(audiopath):
    try:
        speech, rate = librosa.load(audiopath, sr=16000)
        audio_input = whisper_processor(speech, return_tensors="pt", sampling_rate=16000)
        with torch.no_grad():
            generated_ids = whisper_model.generate(audio_input["input_features"])
        return whisper_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    except Exception as e:
        print(f"Error transcribing audio: {e}")
        return None


In [ ]:
# Define multimodal projector class
class ProjectionBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.pre_norm = nn.LayerNorm(input_dim)
        self.proj = nn.Sequential(nn.Linear(input_dim, output_dim), nn.GELU(), nn.Linear(output_dim, output_dim))

    def forward(self, x):
        return self.proj(self.pre_norm(x))

class MultimodalProjector(nn.Module):
    def __init__(self, image_input_dim, audio_input_dim, output_dim):
        super().__init__()
        self.image_proj = ProjectionBlock(image_input_dim, output_dim)
        self.audio_proj = ProjectionBlock(audio_input_dim, output_dim)

    def forward(self, image_embedding=None, audio_embedding=None):
        projected_image = self.image_proj(image_embedding) if image_embedding is not None else None
        projected_audio = self.audio_proj(audio_embedding) if audio_embedding is not None else None
        return projected_image, projected_audio


# Dataset preparation with better error handling and modularization
def prepare_dataset(image_embeddings_path, dataset_path, cache_dir=None):
    image_embeddings = load_embeddings(image_embeddings_path)
    with open(dataset_path, 'r') as f:
        data = json.load(f)
    processed_data = [{"conversation": item["conversations"], "image_embedding": image_embeddings['embeddings'][np.where(image_embeddings['ids'] == item['image'])[0][0]] if image_embeddings and "image" in item else None, "audio_path": item.get("audio")} for item in data]
    dataset = Dataset.from_dict({"conversation": [item["conversation"] for item in processed_data], "image_embedding": [item.get("image_embedding") for item in processed_data], "audio_path": [item.get("audio_path") for item in processed_data]})
    tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct", trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    tokenizer.chat_template = """
    {% for message in messages %}
    {% if message.role == 'system' %}<|system|>{{message.content}}<|endoftext|>{% elif message.role == 'user' %}<|user|>{{message.content}}<|endoftext|>{% elif message.role == 'assistant' %}<|assistant|>{{message.content}}<|endoftext|>{% endif %}{% endfor %}
    """
    prepared_dataset = dataset.map(lambda examples: prepare_example(examples, tokenizer), batched=True, remove_columns=dataset.column_names, batch_size=1).with_format("torch")
    dataset_dict = DatasetDict({"train": prepared_dataset.train_test_split(test_size=0.1)["train"], "test": prepared_dataset.train_test_split(test_size=0.1)["test"]})
    if cache_dir:
        os.makedirs(cache_dir, exist_ok=True)
        dataset_dict.save_to_disk(cache_dir)
    return dataset_dict, tokenizer

# Example preparation for dataset rows
def prepare_example(examples, tokenizer):
    image_embeddings, audio_embeddings, tokenized_inputs = [], [], []
    for idx, conv in enumerate(examples["conversation"]):
        image_embedding = torch.tensor(examples["image_embedding"][idx]) if examples["image_embedding"][idx] is not None else None
        transcription = transcribe_speech(examples["audio_path"][idx]) if "audio_path" in examples and examples["audio_path"][idx] else None
        for i in range(0, len(conv), 2):
            if i + 1 < len(conv):
                human_msg = conv[i]["value"].replace("<image>", "").replace("<audio>", "").strip()
                if transcription:
                    human_msg += f"\nAudio Transcription: {transcription}"
                gpt_msg = conv[i + 1]["value"]
                tokenized_input = tokenizer.apply_chat_template([{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": f"{human_msg}"}, {"role": "assistant", "content": gpt_msg}], return_tensors="pt", padding=True)
                tokenized_inputs.append(tokenized_input.squeeze(0))
                if image_embedding is not None:
                    image_embeddings.append(image_embedding)
    max_length = max(input.shape[0] for input in tokenized_inputs)
    padded_inputs = [torch.nn.functional.pad(input, (0, max_length - input.shape[0])) for input in tokenized_inputs]
    result = {"input_ids": torch.stack(padded_inputs), "attention_mask": torch.stack(padded_inputs).ne(tokenizer.pad_token_id).long(), "labels": torch.stack(padded_inputs).clone()}
    if image_embeddings:
        result["image_embeddings"] = torch.stack(image_embeddings)
    if audio_embeddings:
        result["audio_embeddings"] = torch.stack(audio_embeddings)
    return result


In [ ]:
from torch.utils.checkpoint import checkpoint_sequential

In [ ]:
class Phi3WithProjector(PreTrainedModel):
    def __init__(self, config, phi3_model, projector):
        super().__init__(config)
        self.phi3_model = phi3_model
        self.projector = projector

    def forward(self, input_ids=None, attention_mask=None, inputs_embeds=None, image_embeddings=None, audio_embeddings=None, labels=None, **kwargs):
        # Use get_input_embeddings() to retrieve the embeddings layer
        if inputs_embeds is None:
            inputs_embeds = self.phi3_model.get_input_embeddings()(input_ids)

        # Project both image and audio embeddings to the appropriate dimension
        projected_image, projected_audio = self.projector(image_embeddings, audio_embeddings)

        # Concatenate the embeddings
        embeddings_to_concat = [inputs_embeds]
        if projected_image is not None:
            embeddings_to_concat.append(projected_image.unsqueeze(1))
        if projected_audio is not None:
            embeddings_to_concat.append(projected_audio.unsqueeze(1))

        combined_embeddings = torch.cat(embeddings_to_concat, dim=1)

        # Modify how the attention mask is extended
        extended_attention_mask = attention_mask.clone()  # Start with a copy

        # Extend for image and audio, if present
        if projected_image is not None:
            extended_attention_mask = torch.cat([extended_attention_mask, torch.ones_like(extended_attention_mask[:, :1])], dim=1)
        if projected_audio is not None:
            extended_attention_mask = torch.cat([extended_attention_mask, torch.ones_like(extended_attention_mask[:, :1])], dim=1)

        # Adjust labels to match the extended input sequence length
        if labels is not None:
            # Pad labels with -100 to ignore the added tokens in the loss calculation
            num_added_tokens = sum(1 for emb in [projected_image, projected_audio] if emb is not None)
            labels = torch.cat([labels, torch.full((labels.shape[0], num_added_tokens), -100, dtype=labels.dtype, device=labels.device)], dim=1)

        return self.phi3_model(
            inputs_embeds=combined_embeddings,
            attention_mask=extended_attention_mask,
            labels=labels,
            **kwargs
        )

    def get_input_embeddings(self):
        """Returns the model's input embeddings."""
        return self.phi3_model.get_input_embeddings()

    def set_input_embeddings(self, value):
        """Sets the model's input embeddings."""
        self.phi3_model.set_input_embeddings(value)


In [ ]:
def train_model(dataset_dict, tokenizer, output_dir):
    from transformers import BitsAndBytesConfig

    # Configure quantization
    quantization_config = BitsAndBytesConfig(
        load_in_8bit=True,
        load_in_8bit_fp32_cpu_offload=True
    )
    # Explicitly define device_map to load the model onto the GPU
    device_map = {"": 0}  # Load the model to the first GPU (index 0

    # Load the main model with dynamic quantization configuration
    phi3_model = AutoModelForCausalLM.from_pretrained(
        "microsoft/Phi-3-mini-128k-instruct",
        quantization_config=quantization_config,
        torch_dtype=torch.float16,
        device_map=device_map
    )

    phi3_model = prepare_model_for_kbit_training(phi3_model)

    # Set output_dim dynamically based on the model's hidden size
    output_dim = phi3_model.config.hidden_size

    # Initialize MultimodalProjector with the dynamically set output_dim
    projector = MultimodalProjector(
        image_input_dim=512,     # Replace with actual image embedding dimension if different
        audio_input_dim=512,     # Replace with actual audio embedding dimension if different
        output_dim=output_dim    # Dynamic output dimension based on model's hidden size
    )

    # Initialize Phi3WithProjector model
    model = Phi3WithProjector(phi3_model.config, phi3_model, projector )

    # Configure LoRA
    lora_config = LoraConfig(
        r=8,
        lora_alpha=16,
        target_modules=["self_attn.qkv_proj", "self_attn.o_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type=TaskType.CAUSAL_LM
    )
    model = get_peft_model(model, lora_config)


    # Training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=1/1000,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        learning_rate=2e-4,
        fp16=True,
        save_total_limit=2,
        logging_steps=100,
        save_steps=500,
        eval_steps=500,
        eval_strategy="steps",
        load_best_model_at_end=True,
        remove_unused_columns=False

    )

    # Trainer setup
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset_dict['train'],
        eval_dataset=dataset_dict['test'],
        tokenizer=tokenizer,
        data_collator=MultimodalDataCollator(tokenizer),
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )



    trainer.train()
    trainer.save_model(output_dir)


In [ ]:
if __name__ == "__main__":
    image_embeddings_path = "/content/drive/MyDrive/LLM/Copy of clip_embeddings_instruct150k_15Oct_2024.npz"
    dataset_path = "/content/llava_instruct_150k.json"
    output_dir = "/content/drive/MyDrive/LLM_training_output"
    whisper_model_name = "openai/whisper-small"
    cache_dir = "/content/"
    #dataset_dict, tokenizer = prepare_dataset(image_embeddings_path, dataset_path, cache_dir)
    if os.path.exists(cache_dir):
        dataset_dict = DatasetDict.load_from_disk(cache_dir)
    else:
      dataset_dict, tokenizer = prepare_dataset(image_embeddings_path, dataset_path, cache_dir)


In [ ]:
# Import the EarlyStoppingCallback
from transformers import EarlyStoppingCallback

In [ ]:
train_model(dataset_dict, tokenizer, output_dir)

Unused kwargs: ['load_in_8bit_fp32_cpu_offload']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
